This notebook shows how to use the obtained Random Forest models for each region to get predictions for housing price index (HPI). In particular, I use two IPCC climate model results (RCP4_5, which is the "best case scenario" and RCP8_5, which is the "intermediate case scenario") to extract predictions for temperatures, temperature trends, precipitation rates, and precipitation rate trends for 2027-2030 time period. I plug these predictions into my Random Forest models (keeping all other parameters equal, though they could be updated if I could get data for better predictions) to calculate HPI rates of change. I then compute predictions for "investment profittability" over 2027-2030 time period, which I am measuring as the rate of change of housing price index (HPI) normalized by 2020 housing value within each 3-digit zipcode. The predictions are plotted in static and interactive maps in the "DataPlotting_predictions.ipynb" notebook.

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#Load dataframe with all of the historic data (economic and environmental) 
#    and IPCC climate model predictions for temperature and precipitation rates over 2027-2030 period,
#    tabulated at 3-digit zipcode
master_df = pd.read_pickle('ipcc_master_df.pkl')
master_df.head()

,Zipcode,AnnualTrend_2000_2021,AnnualTrend_2000_2008,AnnualTrend_2015_2021,min_year,max_year,INTPTLAT,INTPTLONG,ALAND_SQMI,AWATER_SQMI,...,RCP4_5_temp_trend,RCP8_5_precip_mean,RCP8_5_precip_trend,RCP8_5_temp_mean,RCP8_5_temp_trend,RCP4_5_HPI_pred,RCP8_5_HPI_pred,Most_profittable_2021,Most_profittable_2030,Most_profittable_2030_8_5
0,010,0.031272,0.086983,0.053891,2000.0,2021.0,42.269354,-72.567846,1276.131,38.492,...,-0.066668,0.000041,-7.986415e-07,12.677643,-0.430109,-0.006974,-0.006974,-0.010183,-0.023522,-0.023218
1,011,0.031176,0.080476,0.075606,2000.0,2021.0,42.106624,-72.548348,40.946,1.716,...,-0.066668,0.000041,-7.986415e-07,12.677643,-0.430109,-0.006974,-0.006974,0.052835,-0.035400,-0.034943
2,012,0.031742,0.091175,0.051823,2000.0,2021.0,42.341925,-73.227852,902.053,16.827,...,-0.076331,0.000039,5.409820e-07,11.607361,-0.410024,-0.006974,-0.006974,-0.013945,-0.019218,-0.018970
3,013,0.031539,0.086385,0.057243,2000.0,2021.0,42.593027,-72.568520,786.429,27.536,...,-0.030959,0.000040,5.224078e-07,10.809631,-0.491792,-0.009275,-0.009275,-0.005033,-0.034272,-0.033830
4,014,0.022781,0.055906,0.076745,2000.0,2021.0,42.589492,-71.794956,474.266,14.954,...,0.009978,0.000039,1.343010e-07,12.504913,-0.514172,0.010629,0.010629,0.055001,0.030149,0.029760


In [3]:
#Load dictionary with states corresponding to each economic region
state_region_dict = pickle.load(open("state_region_dict.pkl", "rb"))
region_list = list(state_region_dict.keys())

In [ ]:
#Now loading each regional model separately, make predictions for changes in HPI over 2027-2030 period

for i in range(len(region_list)):
    #load RandomForest model
    filename = 'RFmodel_'+region_list[i]+'.sav'  
    loaded_model = pickle.load(open(filename,'rb'))
    
    #make predictions for HPI using RCP4_5 climate model ("best case scenario")
    df_sliced = master_df.loc[master_df['region']==region_list[i]]
    df_sliced.loc[:,'LandTemp_Monthly_AVG'] = df_sliced.loc[:,'RCP4_5_temp_mean'] 
    df_sliced.loc[:,'LandTemp_AnnualTrend'] = df_sliced.loc[:,'RCP4_5_temp_trend']
    df_sliced.loc[:,'Precip_AnnualTrend'] = df_sliced.loc[:,'RCP4_5_precip_trend']
    df_sliced.loc[:,'Precip_Monthly_AVG'] = df_sliced.loc[:,'RCP4_5_precip_mean']
    X2 = df_sliced[['DP04_0089E','Owner_Renter_ratio','Median_income','GDP_trend_norm','NO2_Monthly_AVG','VegInd_AnnualTrend','VegInd_Monthly_AVG',
                'LandTemp_AnnualTrend','LandTemp_Monthly_AVG','Precip_AnnualTrend','Precip_Monthly_AVG']]
    result = loaded_model.predict(X2)
    ind=df_sliced.index
    master_df.loc[ind,'RCP4_5_HPI_pred']=result
    
    #make predictions for HPI using RCP8_5 climate model ("intermediate case scenario")
    df_sliced = master_df.loc[master_df['region']==region_list[i]]
    df_sliced.loc[:,'LandTemp_Monthly_AVG'] = df_sliced.loc[:,'RCP8_5_temp_mean'] 
    df_sliced.loc[:,'LandTemp_AnnualTrend'] = df_sliced.loc[:,'RCP8_5_temp_trend']
    df_sliced.loc[:,'Precip_AnnualTrend'] = df_sliced.loc[:,'RCP8_5_precip_trend']
    df_sliced.loc[:,'Precip_Monthly_AVG'] = df_sliced.loc[:,'RCP8_5_precip_mean']
    X2 = df_sliced[['DP04_0089E','Owner_Renter_ratio','Median_income','GDP_trend_norm','NO2_Monthly_AVG','VegInd_AnnualTrend','VegInd_Monthly_AVG',
                'LandTemp_AnnualTrend','LandTemp_Monthly_AVG','Precip_AnnualTrend','Precip_Monthly_AVG']]
    result = loaded_model.predict(X2)
    ind=df_sliced.index
    master_df.loc[ind,'RCP8_5_HPI_pred']=result

In [ ]:
#Calculate "profittability": change in rate of change in HPI normalized by 2020 mean housing price 
#  for each 3-digit zipcode

c1 = (master_df['AnnualTrend_2018_2021']-master_df['AnnualTrend_2000_2008'])/master_df['DP04_0089E']
c2 = (master_df['RCP4_5_HPI_pred'])/master_df['DP04_0089E']
c3 = (master_df['RCP8_5_HPI_pred'])/master_df['DP04_0089E']

master_df['Most_profittable_2021'] = c1/c1.max()
master_df['Most_profittable_2030'] = c2/c2.max()           #"best case scenario" climate model prediction
master_df['Most_profittable_2030_8_5'] = c3/c3.max()       #"intermediate case scenario" climate model prediction

In [ ]:
# Save dataframe (useful for plotting later)
master_df.to_pickle('ipcc_master_df.pkl')
master_df.to_json('predict_df.json',orient='records')

In [ ]:
# Now need to add extra rows to the dataframe so that will be able to plot "All U.S." in region select
# i.e., will need to double the number of rows so that have rows where df['region']=='All U.S.'
# This is necessary to make interactive maps

master_df2 = master_df.copy()
master_df2['region'] = 'All U.S.'
master_df_large = pd.concat((master_df,master_df2))

# Save df as json to be used for Altair plotting later
master_df_large.to_json('predict_df_large.json',orient='records')

In [ ]:
#Now have to add more extra rows to the data frame to do the same as the regions, but for states
# This is necessary to make interactive maps

master_df_large2 = master_df_large.copy()
master_df_large2['state'] = 'All U.S.'
master_df_all = pd.concat((master_df_large,master_df_large2))


In [ ]:
# Also need to record state IDs associated with state names (will need for plotting)
# For (convoluted) plotting reasons, will need both column "id" and column "state_id" with the same information

#Read in dictionary that has all the state names and IDs 
state_dict = pickle.load(open("state_id_dict.pkl", "rb"))

#Now loop over all rows after resetting index
master_df_all = master_df_all.reset_index()
master_df_all['state_id'] = 0
for i in range(len(master_df_all)):
    master_df_all.loc[i,'state_id'] = state_dict[master_df_all.loc[i,'state']]
master_df_all['id'] = master_df_all['state_id']

In [ ]:
# Save df as json to be used for Altair plotting later
master_df_all.to_json('predict_df_all.json',orient='records')